In [ ]:
graph_file = "./output/faq/nx_graph.pkl"
query = "does the phone come with charger?" # "what's included in the box?"

In [41]:
import pickle
import networkx as nx

G = pickle.load(open(graph_file, "rb"))
G

In [ ]:
import networkx as nx
import ollama
import json
from itertools import combinations


with open("./input/config.json", "r") as file:
    config = json.load(file)

model = config['model_name']

system_prompt_root_word = """You are a helpful assistant that extracts the root form of the given word in lowercase.
Return only the root word in response."""

# instead try to find source, relation and target keywords?
system_prompt_key_words = """You are a helpful assistant that extracts keywords from the given question in root form (e.g. ran becomes run) and lowercase.
The returned keywords should be critical to answer the question.
The returned words should be separated by space.
Return only the keywords in response separated by ','"""

def get_llm_response(text, system_prompt):
    response = ollama.chat(model=model, messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": text}
        ])
    # print(f'response={response['message']['content']}')
    resp_content = response['message']['content']
    return resp_content

# --- Step 2: Retrieve relevant triples ---
def search_kg(G, query):
    print(f"no. of edges in KG: {G.number_of_edges()}")
    results = []
    for u, v, d in G.edges(data=True):
        relation = d.get('label', None) if d else None
        relation = d.get('relation', None) if relation is None else relation
        triple = (u, relation, v)
        print(f'triple: {triple}')
        # naive: check if query words appear in triple

        if any(get_llm_response(word, system_prompt_root_word) in " ".join(map(str, triple)).lower() for word in query.split()):
            results.append(triple)
    return results

def search_kg2(G, query):
    keywords = get_llm_response(query, system_prompt_key_words)
    print(f'keywords={keywords}')
    keywords = [part.strip() for part in keywords.split(',')]
    pairs = list(combinations(keywords, 2))
    print(f'pairs={pairs}')
    relations = []
    for u, v in pairs:
        paths = list(nx.all_simple_paths(G, source=u, target=v))
        print(f'paths={paths}')
        for path in paths:
            for i in range(len(path)-1):
                print(f'path{i}[{path[i]},{path[i+1]}]=>{G[path[i]]}-->{G[path[i]][path[i+1]]}')
                for key in G[path[i]][path[i+1]]:
                    print(f'key={key}, value={G[path[i]][path[i+1]][key]}')
                    rel = G[path[i]][path[i+1]][key]['relation']
                relations.append((path[i],rel, path[i+1]))

    print(f'relations={relations}')
    return relations

triples = search_kg2(G, query)
print(f'number of relevant triples found: {len(triples)}, {triples}')

# --- Step 3: Send to Ollama model ---
context = f"""
You are given facts from a knowledge graph:

{triples}

Answer the user query based ONLY on these facts.
Answer in full sentence.
Query: {query}
"""

response = ollama.chat(model="gemma3:4b-it-qat", messages=[{"role": "user", "content": context}])

print(response["message"]["content"])


keywords=phone, charger
pairs=[('phone', 'charger')]
paths=[]
relations=[]
number of relevant triples found: 0, []
I cannot answer the question "does the phone come with charger?" because the provided facts do not contain any information about whether a phone comes with a charger.


In [39]:
list(G.edges(data=True))

[('box', 'Phone', {'relation': 'include'}),
 ('box', 'USB-C cable', {'relation': 'include'}),
 ('box', '30W charger', {'relation': 'include'}),
 ('box', 'SIM eject tool', {'relation': 'include'}),
 ('box', 'quick start guide', {'relation': 'include'}),
 ('box', 'clear case', {'relation': 'include'}),
 ('phone', 'setup wizard', {'relation': 'set_up'}),
 ('phone', 'Wi-Fi', {'relation': 'connect'}),
 ('phone', 'cloud account', {'relation': 'sign_in'}),
 ('phone', 'screen lock', {'relation': 'set'}),
 ('phone', 'data', {'relation': 'restore'}),
 ('phone', 'battery', {'relation': 'last'}),
 ('phone', '30W charger', {'relation': 'charge_with'}),
 ('phone', 'water', {'relation': 'be_resistant'}),
 ('phone', 'dust', {'relation': 'be_resistant'}),
 ('phone', 'microSD', {'relation': 'support'}),
 ('phone', 'cloud backup', {'relation': 'have'}),
 ('phone', 'software update policy', {'relation': 'have'}),
 ('phone', 'dual SIM', {'relation': 'support'}),
 ('phone', 'eSIM', {'relation': 'support'}),